In [2]:
# 0. Setup
import duckdb
import pandas as pd
from pathlib import Path

# If your package is importable, you can do:
# from faers_signals.config import WAREHOUSE_DB_PATH
# Otherwise, just hardcode the path for now:
PROJECT_ROOT = Path.cwd().parents[0]  # adjust if needed
WAREHOUSE_DB_PATH = PROJECT_ROOT / "data" / "warehouse" / "faers_2019_present.duckdb"

print("DB path:", WAREHOUSE_DB_PATH)
con = duckdb.connect(str(WAREHOUSE_DB_PATH))


DB path: /shared/crollo/faers_signals/data/warehouse/faers_2019_present.duckdb


In [3]:
# 1. List all tables in the warehouse
tables_df = con.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'main'
    ORDER BY table_name
""").fetchdf()

tables_df

,table_name
0,demo
1,demo_dedup
2,drug
3,drug_dedup
4,indi
5,indi_dedup
6,outc
7,outc_dedup
8,primaryid_kept
9,reac


In [4]:
# 1b. Row counts per table (quick sanity check)
row_counts = []
for table in tables_df["table_name"]:
    n = con.execute(f"SELECT COUNT(*) AS n FROM {table}").fetchone()[0]
    row_counts.append({"table": table, "n_rows": n})

row_counts_df = pd.DataFrame(row_counts).sort_values("n_rows", ascending=False)
row_counts_df


,table,n_rows
2,drug,52191794
3,drug_dedup,39469240
9,reac,39462044
4,indi,34053500
10,reac_dedup,29751480
5,indi_dedup,26961769
13,ther,17831837
14,ther_dedup,13184186
0,demo,11697906
8,primaryid_kept,9884211


In [5]:
# 2.1 DEMO schema
con.execute("DESCRIBE demo").fetchdf()


,column_name,column_type,null,key,default,extra
0,primaryid,VARCHAR,YES,None,None,None
1,caseid,VARCHAR,YES,None,None,None
2,caseversion,VARCHAR,YES,None,None,None
3,i_f_code,VARCHAR,YES,None,None,None
4,event_dt,VARCHAR,YES,None,None,None
5,mfr_dt,VARCHAR,YES,None,None,None
6,init_fda_dt,VARCHAR,YES,None,None,None
7,fda_dt,VARCHAR,YES,None,None,None
8,rept_cod,VARCHAR,YES,None,None,None
9,auth_num,VARCHAR,YES,None,None,None


In [6]:
# 2.1b Sample rows from DEMO
con.execute("""
    SELECT *
    FROM demo
    LIMIT 5
""").fetchdf()


,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country,year,quarter
0,1000661812,10006618,12,F,None,20190301,20140313,20190311,PER,None,...,Y,None,None,20190311,None,MD,US,US,2019,Q1
1,1000808590,10008085,90,F,20131213,20190129,20140313,20190212,EXP,None,...,Y,50.34,KG,20190212,None,MD,CA,CA,2019,Q1
2,100107484,10010748,4,F,2014,20190121,20140314,20190128,EXP,None,...,Y,None,None,20190128,None,MD,US,US,2019,Q1
3,100157133,10015713,3,F,None,20190206,20140317,20190215,PER,None,...,Y,71.2,KG,20190215,None,CN,US,US,2019,Q1
4,100171328,10017132,8,F,20140908,20190219,20140318,20190228,EXP,None,...,Y,None,None,20190228,None,CN,AR,AR,2019,Q1


In [7]:
# 2.2 DRUG schema
con.execute("DESCRIBE drug").fetchdf()


,column_name,column_type,null,key,default,extra
0,primaryid,VARCHAR,YES,None,None,None
1,caseid,VARCHAR,YES,None,None,None
2,drug_seq,VARCHAR,YES,None,None,None
3,role_cod,VARCHAR,YES,None,None,None
4,drugname,VARCHAR,YES,None,None,None
5,prod_ai,VARCHAR,YES,None,None,None
6,val_vbm,VARCHAR,YES,None,None,None
7,route,VARCHAR,YES,None,None,None
8,dose_vbm,VARCHAR,YES,None,None,None
9,cum_dose_chr,VARCHAR,YES,None,None,None


In [8]:
# 2.3 REAC schema
con.execute("DESCRIBE reac").fetchdf()


,column_name,column_type,null,key,default,extra
0,primaryid,VARCHAR,YES,None,None,None
1,caseid,VARCHAR,YES,None,None,None
2,pt,VARCHAR,YES,None,None,None
3,drug_rec_act,VARCHAR,YES,None,None,None
4,year,BIGINT,YES,None,None,None
5,quarter,VARCHAR,YES,None,None,None


In [9]:
# 2.3b Sample rows from REAC
con.execute("""
    SELECT *
    FROM reac
    LIMIT 5
""").fetchdf()


,primaryid,caseid,pt,drug_rec_act,year,quarter
0,1000661812,10006618,Drug hypersensitivity,None,2019,Q1
1,1000661812,10006618,Muscular weakness,None,2019,Q1
2,1000661812,10006618,Rash,None,2019,Q1
3,1000661812,10006618,Urticaria,None,2019,Q1
4,1000808590,10008085,Abdominal adhesions,None,2019,Q1


In [10]:
# 3. Distinct year/quarter combinations in DEMO
con.execute("""
    SELECT year, quarter, COUNT(*) AS n_reports
    FROM demo
    GROUP BY year, quarter
    ORDER BY year, quarter
""").fetchdf()


,year,quarter,n_reports
0,2019,Q1,413734
1,2019,Q2,441108
2,2019,Q3,452873
3,2019,Q4,419581
4,2020,Q1,460327
5,2020,Q2,429227
6,2020,Q3,431667
7,2020,Q4,436148
8,2021,Q1,463741
9,2021,Q2,479945


In [11]:
# 4.1 Missing primaryid / caseid in DEMO
con.execute("""
    SELECT
        SUM(CASE WHEN primaryid IS NULL OR TRIM(primaryid) = '' THEN 1 ELSE 0 END) AS missing_primaryid,
        SUM(CASE WHEN caseid IS NULL OR TRIM(caseid) = '' THEN 1 ELSE 0 END) AS missing_caseid,
        COUNT(*) AS total
    FROM demo
""").fetchdf()


,missing_primaryid,missing_caseid,total
0,0.0,0.0,11697906


In [12]:
# 4.2 How many primaryids appear more than once in DEMO?
con.execute("""
    SELECT COUNT(*) AS n_with_duplicates
    FROM (
        SELECT primaryid, COUNT(*) AS c
        FROM demo
        GROUP BY primaryid
        HAVING COUNT(*) > 1
    )
""").fetchdf()


,n_with_duplicates
0,204


In [13]:
# 5.1 Pick a random primaryid from DEMO
random_id = con.execute("""
    SELECT primaryid
    FROM demo
    USING SAMPLE 1
""").fetchone()[0]

random_id


'161711201'

In [14]:
# 5.2 Show DEMO + all DRUG + all REAC rows for that primaryid
demo_sample = con.execute(f"""
    SELECT *
    FROM demo
    WHERE primaryid = '{random_id}'
""").fetchdf()

drug_sample = con.execute(f"""
    SELECT *
    FROM drug
    WHERE primaryid = '{random_id}'
""").fetchdf()

reac_sample = con.execute(f"""
    SELECT *
    FROM reac
    WHERE primaryid = '{random_id}'
""").fetchdf()

demo_sample, drug_sample, reac_sample


(   primaryid    caseid caseversion i_f_code  event_dt    mfr_dt init_fda_dt  \
 0  161711201  16171120           1        I  20190404  20190404    20190408   
 
      fda_dt rept_cod auth_num  ... e_sub    wt wt_cod   rept_dt to_mfr  \
 0  20190408      PER     None  ...     Y  None   None  20190408   None   
 
   occp_cod reporter_country occr_country  year quarter  
 0       CN               US           US  2019      Q2  
 
 [1 rows x 27 columns],
    primaryid    caseid drug_seq role_cod drugname   prod_ai val_vbm    route  \
 0  161711201  16171120        1       PS  ELIQUIS  APIXABAN       1  Unknown   
 
   dose_vbm cum_dose_chr  ... rechal   lot_num exp_dt nda_num dose_amt  \
 0      UNK         None  ...   None   UNKNOWN   None  202155     None   
 
   dose_unit dose_form dose_freq  year quarter  
 0      None      None      None  2019      Q2  
 
 [1 rows x 22 columns],
    primaryid    caseid        pt drug_rec_act  year quarter
 0  161711201  16171120  Headache         Non

In [15]:
con.execute("""
    SELECT sex, COUNT(*) AS n
    FROM demo
    GROUP BY sex
    ORDER BY n DESC
""").fetchdf()


,sex,n
0,F,5899218
1,M,4036022
2,None,1746270
3,UNK,16056
4,P,158
5,I,109
6,T,73


In [16]:
con.execute("""
    SELECT
        CASE
            WHEN TRY_CAST(age AS DOUBLE) IS NULL THEN 'missing/invalid'
            WHEN TRY_CAST(age AS DOUBLE) < 18 THEN '<18'
            WHEN TRY_CAST(age AS DOUBLE) BETWEEN 18 AND 44 THEN '18–44'
            WHEN TRY_CAST(age AS DOUBLE) BETWEEN 45 AND 64 THEN '45–64'
            WHEN TRY_CAST(age AS DOUBLE) >= 65 THEN '65+'
        END AS age_group,
        COUNT(*) AS n
    FROM demo
    GROUP BY age_group
    ORDER BY n DESC
""").fetchdf()


,age_group,n
0,missing/invalid,5051484
1,65+,2525885
2,45–64,2238492
3,18–44,1379899
4,<18,502146


In [17]:
# 7.1 Look at top 20 drugnames by frequency
top_drugs = con.execute("""
    SELECT UPPER(TRIM(drugname)) AS drugname_norm,
           COUNT(*) AS n
    FROM drug
    GROUP BY drugname_norm
    ORDER BY n DESC
    LIMIT 20
""").fetchdf()

top_drugs


,drugname_norm,n
0,ZANTAC,1007462
1,INFLECTRA,740522
2,DUPIXENT,576886
3,HUMIRA,458367
4,REVLIMID,429975
5,RANITIDINE.,403574
6,PREDNISONE,386437
7,METHOTREXATE,333696
8,VEDOLIZUMAB,311093
9,MOUNJARO,296386


In [18]:
drug_of_interest = "ATORVASTATIN"

con.execute(f"""
    SELECT year, quarter, COUNT(*) AS n
    FROM drug
    WHERE UPPER(TRIM(drugname)) = '{drug_of_interest}'
    GROUP BY year, quarter
    ORDER BY year, quarter
""").fetchdf()


,year,quarter,n
0,2019,Q1,6884
1,2019,Q2,7808
2,2019,Q3,8446
3,2019,Q4,7966
4,2020,Q1,8292
5,2020,Q2,7214
6,2020,Q3,7516
7,2020,Q4,7619
8,2021,Q1,7088
9,2021,Q2,6960


In [19]:
con.execute(f"""
    SELECT r.pt AS reaction_pt, COUNT(*) AS n
    FROM reac r
    JOIN drug d USING(primaryid)
    WHERE UPPER(TRIM(d.drugname)) = '{drug_of_interest}'
    GROUP BY r.pt
    ORDER BY n DESC
    LIMIT 20
""").fetchdf()


,reaction_pt,n
0,Fatigue,15093
1,Diarrhoea,12785
2,Off label use,12281
3,Nausea,11736
4,Dyspnoea,11362
5,Acute kidney injury,10456
6,Drug ineffective,9641
7,Dizziness,9523
8,Pain,9492
9,Headache,8675


In [20]:
# 8.1 Compare row counts before/after dedup for demo
con.execute("""
    SELECT 'demo' AS table, COUNT(*) AS n FROM demo
    UNION ALL
    SELECT 'demo_dedup' AS table, COUNT(*) AS n FROM demo_dedup
""").fetchdf()


,table,n
0,demo,11697906
1,demo_dedup,9884211


In [21]:
# 8.2 Similarly for drug and reac
con.execute("""
    SELECT 'drug' AS table, COUNT(*) AS n FROM drug
    UNION ALL
    SELECT 'drug_dedup' AS table, COUNT(*) AS n FROM drug_dedup
""").fetchdf()


,table,n
0,drug,52191794
1,drug_dedup,39469240


In [22]:
con.execute("""
    SELECT 'reac' AS table, COUNT(*) AS n FROM reac
    UNION ALL
    SELECT 'reac_dedup' AS table, COUNT(*) AS n FROM reac_dedup
""").fetchdf()


,table,n
0,reac,39462044
1,reac_dedup,29751480
